<a href="https://colab.research.google.com/github/Asritha0606/GenerativeAI/blob/main/CrewAI_Automating_Event_Planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use case- Automating Event Planning

1. Venue Coordinator
2. Logistics Manager
3. Marketing and Communications Agent


Takeaways-

- Allow Human Feedback
- Getting curated files directly as output
- Serper API for Google search

In [1]:
!pip install -q crewai 'crewai[tools]' --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.52.4 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.


Crew AI Tool: https://docs.crewai.com/concepts/tools

In [21]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew

In [22]:
import os
from google.colab import userdata
groq_api_key = userdata.get('groq_api')
os.environ["GROQ_API_KEY"] = groq_api_key

# hf_api_key = userdata.get('huggingfacehub')
# os.environ["HUGGINGFACEHUB_API_KEY"] = hf_api_key

serper_api_key = userdata.get('serper_api')
os.environ["SERPER_API_KEY"] = serper_api_key

gemini_api_key = userdata.get('Gemini_api')
os.environ["GEMINI_API_KEY"] = gemini_api_key

together_api_key = userdata.get('together_api')
os.environ["TOGETHER_API_KEY"] = together_api_key

In [23]:
from crewai import LLM
llm = LLM(model="gemini/gemini-1.5-flash", api_key=gemini_api_key)

In [ ]:
# from crewai import Agent, LLM
# llm = LLM(
#     model= "together_ai/meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
#     temperature=0,
#     api_key=together_api_key
# )


## Automate Event Planning



Tool Creation

In [24]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

**Agent 1: Venue Coordinator**

In [25]:
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    verbose=False,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=llm, #gem_llm llm
    tools= [search_tool, scrape_tool] #[SerperDevTool(), ScrapeWebsiteTool()]
)

 **Agent 2: Logistics Manager**

In [26]:
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipment"
    ),
    verbose=False,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=llm, #gem_llm llm
    tools= [search_tool, scrape_tool] #[SerperDevTool(), ScrapeWebsiteTool()],
)

**Agent 3: Marketing and Communications Agent**

In [27]:
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    verbose=False,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=llm,  #llm, #groq_llm, gem_llm
    tools= [search_tool, scrape_tool] #[SerperDevTool(), ScrapeWebsiteTool()]
  )

In [28]:
from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks

Task #1: Venue Task

In [29]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

Task #2: Logistic Task

In [30]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

Task #3: Marketing Task

In [31]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    # async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

**Define the crew with agents and tasks**

In [32]:
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=False
)

In [33]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [34]:
import json

# Convert dictionary to JSON string
event_details_str = json.dumps(event_details)

In [35]:
result = event_management_crew.kickoff(inputs=event_details)

 Received None or empty response from LLM call.
 An unknown error occurred. Please check the details below.
 Error details: Invalid response from LLM call - None or empty.
 An unknown error occurred. Please check the details below.
 Error details: Invalid response from LLM call - None or empty.
 ## Final Result: {"name": "Moscone Center", "address": "747 Howard St, San Francisco, CA 94103", "capacity": 4740, "booking_status": "Unknown"}
 

=====
## HUMAN FEEDBACK: Provide feedback on the Final Result and Agent's actions.
Please follow these guidelines:
 - If you are happy with the result, simply hit Enter without typing anything.
 - Otherwise, provide specific improvement requests.
 - You can provide multiple rounds of feedback until satisfied.
=====

i need location venue to be hyderabad
 
Processing your feedback...
 ## Final Result: {"name": "Hyderabad International Convention Centre (HICC)", "address": "Novotel Hyderabad Convention Centre, HITEC City, Hyderabad, Telangana 500081, I

In [36]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'Novotel Hyderabad Convention Centre, HITEC City, Hyderabad, '
            'Telangana 500081, India',
 'booking_status': 'Unknown',
 'capacity': 5800,
 'name': 'Hyderabad International Convention Centre (HICC)'}
